In [0]:
# src/notebooks/ingest_crime_data.py

# 从bundle变量读取环境配置
catalog = dbutils.widgets.get("catalog")
schema = dbutils.widgets.get("schema")
container = dbutils.widgets.get("container")
storage_account = dbutils.widgets.get("storage_account")

# 构建路径
source_path = f"abfss://{container}@{storage_account}.dfs.core.windows.net/chicago_crime/"
checkpoint_path = f"abfss://{container}@{storage_account}.dfs.core.windows.net/_checkpoints/chicago_crime/"

# AutoLoader 读取
df = (spark.readStream
    .format("cloudFiles")
    .option("cloudFiles.format", "csv")
    .option("cloudFiles.schemaLocation", f"{checkpoint_path}/schema")
    .option("cloudFiles.inferColumnTypes", "true")
    .load(source_path)
)

# 写入 Delta Table
(df.writeStream
    .format("delta")
    .option("checkpointLocation", f"{checkpoint_path}/data")
    .trigger(availableNow=True)
    .toTable(f"{catalog}.{schema}.crime_data")
)